In [1]:
#Importing relevant libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

In [9]:
train=pd.read_csv("train.csv")
test=pd.read_csv("Test_fCbTej3_0j1gHmj.csv")
train_path="C:/Users/anant/Desktop/Coding Ground/Data Science/Projects/Untitled Folder/Images/train/"
test_path="C:/Users/anant/Desktop/Coding Ground/Data Science/Projects/Untitled Folder/Images/test/"

In [10]:
# loading training images
train_img = []
for img_name in tqdm(train['filename']):
    # defining the image path
    image_path = train_path + img_name
    # reading the image
    img = imread(image_path)
    # normalizing the pixel values
    img = img/255
    # resizing the image to (28,28,1)
    img = resize(img, output_shape=(28,28,1), mode='constant', anti_aliasing=True)
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    train_img.append(img)

# converting the list to numpy array
train_x = np.array(train_img)
train_x.shape

100%|███████████████████████████████████████████████████████████████████████████| 49000/49000 [01:31<00:00, 534.07it/s]


(49000, 28, 28, 1)

In [12]:
# loading training images
test_img = []
for img_name in tqdm(test['filename']):
    # defining the image path
    image_path = test_path + img_name
    # reading the image
    img = imread(image_path)
    # normalizing the pixel values
    img = img/255
    # resizing the image to (28,28,1)
    img = resize(img, output_shape=(28,28,1), mode='constant', anti_aliasing=True)
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    test_img.append(img)

# converting the list to numpy array
test_x = np.array(test_img)
test_x.shape

100%|███████████████████████████████████████████████████████████████████████████| 21000/21000 [00:42<00:00, 491.40it/s]


(21000, 28, 28, 1)

In [13]:
np.save('X_train.npy', train_x)
np.save('X_test.npy', test_x)

In [14]:
x_train = np.load('X_train.npy', allow_pickle=True)
x_test = np.load('X_test.npy', allow_pickle=True)

In [15]:
print(x_train.shape)
print(x_test.shape)

(49000, 28, 28, 1)
(21000, 28, 28, 1)


In [16]:
y_train = train['label']
from tensorflow.keras.utils import to_categorical
print(y_train.shape)
y_train = to_categorical(y_train, num_classes=10)
print(y_train.shape)
print(y_train[0])

(49000,)
(49000, 10)
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [17]:
model=Sequential()
#adding convolution layer
model.add(Conv2D(10,(3,3),activation='relu',input_shape=(28,28,1)))
#adding pooling layer
model.add(MaxPool2D(2,2))
#adding fully connected layer
model.add(Flatten())
model.add(Dense(50,activation='relu'))
#adding output layer
model.add(Dense(10,activation='softmax'))

In [19]:
#compiling the model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#fitting the model
model.fit(x_train,y_train,epochs=10)

Epoch 1/10
1532/1532 [==============================] - 11s 7ms/step - loss: 0.3525 - accuracy: 0.8985
Epoch 2/10
1532/1532 [==============================] - 12s 8ms/step - loss: 0.1465 - accuracy: 0.9571
Epoch 3/10
1532/1532 [==============================] - 11s 7ms/step - loss: 0.0994 - accuracy: 0.9696
Epoch 4/10
1532/1532 [==============================] - 12s 8ms/step - loss: 0.0740 - accuracy: 0.9781
Epoch 5/10
1532/1532 [==============================] - 11s 7ms/step - loss: 0.0603 - accuracy: 0.9809
Epoch 6/10
1532/1532 [==============================] - 10s 7ms/step - loss: 0.0505 - accuracy: 0.9846
Epoch 7/10
1532/1532 [==============================] - 9s 6ms/step - loss: 0.0419 - accuracy: 0.9873
Epoch 8/10
1532/1532 [==============================] - 9s 6ms/step - loss: 0.0371 - accuracy: 0.9886
Epoch 9/10
1532/1532 [==============================] - 10s 7ms/step - loss: 0.0300 - accuracy: 0.9905
Epoch 10/10
1532/1532 [==============================] - 10s 7ms/step - los

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 10)        100       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 10)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 1690)              0         
                                                                 
 dense (Dense)               (None, 50)                84550     
                                                                 
 dense_1 (Dense)             (None, 10)                510       
                                                                 
Total params: 85,160
Trainable params: 85,160
Non-trainable params: 0
____________________________________________________

In [22]:
submissions = pd.read_csv('sample_submission_npBPSZB.csv')

In [36]:
#Prediction
pred =np.asarray(model.predict(x_test))
predictions = np.argmax(pred,axis=1)

In [37]:
predictions

array([4, 0, 9, ..., 6, 6, 2], dtype=int64)

In [38]:
submissions['label'] = predictions

In [39]:
pd.DataFrame(submissions, columns=['filename','label']).to_csv('sample.csv')